<a href="https://colab.research.google.com/github/pagocro/pagocro/blob/main/bot_periodico_fisso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install python-telegram-bot==13.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from telegram import InputMediaPhoto, InputMediaVideo
from telegram.ext import Updater, CommandHandler,MessageHandler,Filters
from glob import glob
from random import choice, sample
import os
import schedule
import time

# TOKEN del Bot Telegram
TOKEN = "6101526654:AAGhCkv2M9H4AQXf5bMrVr3T0Y_YeFELVgA"

# Percorso della cartella delle immagini
IMAGES_FOLDER = "/content/drive/MyDrive/bot/images"

# Percorso della cartella dei video
VIDEOS_FOLDER = "/content/drive/MyDrive/bot/videos"

# Parole chiave e sinonimi per le foto
PHOTO_KEYWORDS = ["photo", "picture", "photos", "image", "snapshot", "photograph", "shot", "pic", "print", "portrait"]

# Parole chiave e sinonimi per i video
VIDEO_KEYWORDS = ["video", "clip", "movie", "film", "footage"]

# Link a patpassion
PATPASSION_LINKS = {
    "Twitter": "https://twitter.com/PatPassion1",
    "Onlyfans": "https://t.co/ZEdaX2aMuL",
    "Manyvids": "https://t.co/4TaBlWHTQA",
    "Skyprivate": "https://t.co/lyZiKKA7Tu",
    "Cammodels": "https://www.cammodels.com/cam/aslasta811",
    "Stripchat": "https://stripchat.com/PatPassion"
}

def start(update, context):
    message = "Hello my love! I'm Patpassion bot. Here are the available commands:\n\n" \
              "/photo or its synonyms: Send a random photo.\n" \
              "/video or its synonyms: Send a random video.\n" \
              "/photos or its synonyms: Send 5 random photos.\n" \
              "/info or /links: Get links to PatPassion profiles.\n\n" \
              "Feel free to ask me anything or use the commands and kiss"
    update.message.reply_text(message)


class CustomContext:
    def __init__(self, job_queue, bot):
        self.job_queue = job_queue
        self.bot = bot

def send_random_photo(context):
    photo_path = choice(glob(os.path.join(IMAGES_FOLDER, "*")))
    context.job.context.bot.send_photo(chat_id=context.job.context.chat_id, photo=open(photo_path, 'rb'))

def send_random_video(context):
    video_path = choice(glob(os.path.join(VIDEOS_FOLDER, "*")))
    context.job.context.bot.send_video(chat_id=context.job.context.chat_id, video=open(video_path, 'rb'))

def send_random_photo_2(update, context):
    photo_path = choice(glob(os.path.join(IMAGES_FOLDER, "*")))
    context.bot.send_photo(chat_id=update.effective_chat.id, photo=open(photo_path, 'rb'))

def send_random_video_2(update, context):
    video_path = choice(glob(os.path.join(VIDEOS_FOLDER, "*")))
    context.bot.send_video(chat_id=update.effective_chat.id, video=open(video_path, 'rb'))

def schedule_commands(context):
    context.job_queue.run_repeating(send_random_photo, interval=2000, context=context)
    context.job_queue.run_repeating(send_random_video, interval=3000, context=context)
    context.job_queue.run_repeating(send_random_photo, interval=5000, context=context)

def get_patpassion_links(update, context):
    message = "Here are the links to PatPassion profiles:\n\n"
    for name, link in PATPASSION_LINKS.items():
        message += f"{name}: {link}\n"
    update.message.reply_text(message)

def send_multiple_photos(update, context):
    photo_paths = sample(glob(os.path.join(IMAGES_FOLDER, "*")), 5)
    media = [InputMediaPhoto(open(path, 'rb')) for path in photo_paths]
    context.bot.send_media_group(chat_id=update.effective_chat.id, media=media)

def chat_reply(update, context):
    message = update.message.text.lower()

    if any(keyword in message for keyword in PHOTO_KEYWORDS):
        if any(keyword in message for keyword in ["photos", "group", "multiple", "many"]):
            send_multiple_photos(update, context)
        else:
            send_random_photo_2(update, context)
    elif any(keyword in message for keyword in VIDEO_KEYWORDS):
        send_random_video_2(update, context)
    elif message.startswith("/info") or message.startswith("/links"):
        get_patpassion_links(update, context)
    else:
        update.message.reply_text("I'm sorry, I didn't understand that command.")


def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher
    job_queue = updater.job_queue

    context = CustomContext(job_queue, updater.bot)
    context.chat_id = "-1001956603136"  # Inserisci l'ID del chat in cui si desidera inviare le immagini o i video

    schedule_commands(context)

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text, chat_reply))


    updater.start_polling()
    print("Bot started.")
    updater.idle()

if __name__ == '__main__':
    main()


Bot started.
